# **Lung-Nodule-Detection: Part-2 Model Training**

## 目标

在Part-2部分，分别训练两个模型（ **结节分割模型** + **结节分类模型** ），并集成两个模型形成完整的 **端到端的结节检测应用** 。具体目标如下：

- 掌握 ***CNN*** 架构的原理，并在实践中应用到分类任务（结节分类模型）
- 理解 ***U-Net*** 架构背后的设计思想，并在实践中应用到分割任务 （结节分割模型）
- 通过集成两个较小模型实现完整的端到端的应用，体会 **模块化设计** 的一般性原则在深度学习领域的适用性

## 内容

本节主要由3部分组成

- 训练一个分割模型，并展示分割模型的效果
- 训练一个分类模型，并展示分类模型的效果
- 补充必要的步骤，串联分割+分类模型，展示完整的端到端的结节检测应用的效果

## 设置
**必须**先执行一次设置代码块

请先**满足**以下条件：
- 下载 [LUNA2016 Dataset](https://luna16.grand-challenge.org/Download/)
- 解压下载后的数据集到 ***data-unversioned/luna*** 目录下（可不必下载全部subset）

In [1]:
import os
import sys
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# 获取当前工作目录
current_path = os.getcwd()

# 获取当前目录的最后一个字符串（即最后一个文件夹名）
last_directory = os.path.basename(current_path)

# 检查是否是 notebook
if last_directory == 'notebooks':
    # 切换到上一级目录
    os.chdir('..')

#print(f"切换到目录: {os.getcwd()}")

sys.path.append('src')  # 将模块所在的路径添加到 sys.path

from app.cache.prepcache_cls import ClsPrepCacheApp
from app.cache.prepcache_seg import SegPrepCacheApp
from app.train.training_cls import ClassificationTrainingApp
from app.train.training_seg import SegmentationTrainingApp
from app.infer.nodule_analysis import NoduleAnalysisApp
from util.logconf import logging
from app.debug.test_cls import TestClsApp
from util.vis import *

log = logging.getLogger('nb')

2025-05-05 13:14:20.057476: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-05 13:14:20.057535: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-05 13:14:20.058377: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-05 13:14:20.065032: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-05 13:14:21.693672: W tensorflow/compiler/tf2

In [2]:
# 设置随机数种子方便复现实验结果
def set_random_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    #torch.set_num_threads(1)

set_random_seed(42)

In [3]:
# 定义一个辅助函数用于启动应用
def run(app, *argv):
    argv = list(argv)
    log.info("Running: {}({!r}).main()".format(app.__name__, argv))
    app(argv).main()
    log.info("Finished: {}.{!r}).main()".format(app.__name__, argv))

# 训练分割模型

### ***SegPrepCacheApp*** 简介

***SegPrepCacheApp*** 的实现位于 ***src/app/cache/prepcache_seg.py***

为提高训练效率，将 **数据缓存到磁盘** 。  
训练前，请确保已经执行过一次 ***SegPrepCacheApp***

In [4]:
run(SegPrepCacheApp, "--num-workers=2")

2025-04-30 17:07:26,777 INFO     pid:1757807 nb:004:run Running: SegPrepCacheApp(['--num-workers=2']).main()
2025-04-30 17:07:26,779 INFO     pid:1757807 app.cache.prepcache_seg:048:main Starting SegPrepCacheApp, Namespace(batch_size=1024, num_workers=2)
2025-04-30 17:07:29,784 WARNING  pid:1757807 util.util:221:enumerateWithEstimate Stuffing cache ----/538, starting
2025-04-30 17:09:28,966 INFO     pid:1757807 util.util:238:enumerateWithEstimate Stuffing cache   16/538, done at 2025-04-30 18:11:18, 1:03:36
2025-04-30 17:15:40,775 INFO     pid:1757807 util.util:238:enumerateWithEstimate Stuffing cache   64/538, done at 2025-04-30 18:15:34, 1:07:51
2025-04-30 17:41:55,412 INFO     pid:1757807 util.util:238:enumerateWithEstimate Stuffing cache  256/538, done at 2025-04-30 18:19:38, 1:11:55
2025-04-30 18:25:51,243 WARNING  pid:1757807 util.util:251:enumerateWithEstimate Stuffing cache ----/538, done at 2025-04-30 18:25:51
2025-04-30 18:25:51,270 INFO     pid:1757807 nb:006:run Finished: S

### ***SegmentationTrainingApp*** 简介

***SegmentationTrainingApp*** 的实现位于 ***src/app/train/training_seg.py***

#### **Model 架构**

分割任务主体部分采用成熟的 ***U-Net*** 网络架构，并用一个 ***UNetWrapper*** 类封装起来，针对肺结节分割任务的特定需求作些修改。

***U-Net*** 的 **核心设计理念** ：
- 由 **编码器（下采样）** 和 **解码器（上采样）** 两部分组成
- 编码器：逐步 **下采样（用卷积+池化）** ，提取高层次的语义特征
- 解码器：逐步 **上采样（用转置卷积或上采样+卷积）** ，恢复空间分辨率，生成像素级别的预测
- **跳跃连接**：每一层的编码器输出，直接连接到对应解码器层。这样 **既保留了局部细节（低层特征），又利用了全局语义信息（高层特征）** 。

更多关于 ***U-Net*** 的信息可参考 https://arxiv.org/abs/1505.04597 和 https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/

***U-Net*** 实现位于 ***src/core/model/unet.py***

***UNetWrapper*** 是对 ***U-Net*** 的封装，针对结节分割应用的需要作了调整，实现位于 ***src/core/model/model_seg.py***
```
class UNetWrapper(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()

        self.input_batchnorm = nn.BatchNorm2d(kwargs['in_channels'])
        self.unet = UNet(**kwargs)
        self.final = nn.Sigmoid()
#......................................
```

#### **输入/输出**

输入：

- *3D* 的CT影像数据在结节中心位置处的 *2D* 切片（若以病人坐标系（xyz）来看，为z轴切片，即每个切片为x-y平面）。

- 同时，取 **上下相邻的3个切片** 作为不同的 **数据通道** 。

- 训练时，先取出 *(C=7, H=96, W=96)* 的以结节为中心的数据块，同时在96x96的范围内，随机裁剪出64x64的区域，形状为 ***(C=7, H=64, W=64)*** 。

- 验证时，以整个完整切片输入，形状为 ***(C=7, H=512, W=512)*** 。

输出：形如 ***(C=1, H_o=H_in, W_o=W_in)*** ，每个点表示该点为结节的概率。

#### **设计权衡**

CT影像是 *3D* 数据，可以对传统的 *U-Net* 进行修改，以适应 *3D* 数据的输入，但这样做在一般性能的机器上会导致 **显存不够用**。

本项目采用的取上下切片作为不同数据通道的方案， **虽然不能完全学习到结节在z方向的空间特性，但也不至于完全丢失相邻切片间的关联性** ，是个折衷的权衡。

而且，由于结节通常较小，而z方向的分辨率一般较大，取上下各3个相邻切片应足以覆盖。

In [4]:
run(SegmentationTrainingApp, "--num-workers=1", "--epochs=100", "--augmented", "--tb-prefix=seg", "seg")

2025-04-30 18:55:29,015 INFO     pid:1776838 nb:004:run Running: SegmentationTrainingApp(['--num-workers=1', '--epochs=100', '--augmented', '--tb-prefix=seg', 'seg']).main()
2025-04-30 18:55:29,093 INFO     pid:1776838 app.train.training_seg:148:initModel Using CUDA; 1 devices.
2025-04-30 18:55:30,194 INFO     pid:1776838 app.train.training_seg:212:main Starting SegmentationTrainingApp, Namespace(batch_size=16, num_workers=1, epochs=100, augmented=True, augment_flip=False, augment_offset=False, augment_scale=False, augment_rotate=False, augment_noise=False, tb_prefix='seg', comment='seg')
2025-04-30 18:55:35,943 INFO     pid:1776838 core.dsets_seg:273:__init__ <core.dsets_seg.TrainingLuna2dSegmentationDataset object at 0x7fbcfff8e880>: 799 training series, 8343 slices, 1028 nodules
2025-04-30 18:55:36,016 INFO     pid:1776838 core.dsets_seg:273:__init__ <core.dsets_seg.Luna2dSegmentationDataset object at 0x7fbcfff8e970>: 89 validation series, 1122 slices, 154 nodules
2025-04-30 18:55:3

#### **基于 Tensorboard 的效果展示**

完整的 **Metrics** 见 ***runs/seg/2025-04-30_18.55.29_trn_seg_seg/*** 和 ***runs/seg/2025-04-30_18.55.29_val_seg_seg/***

**训练集** 数据标注为 **绿色**  
**验证集** 数据标注为 **白色**

***Loss*** **趋势图**

![seg-loss.png](../images/seg-loss.png)

从 ***loss*** 来看，  
**训练集** 一直呈 **下降** 趋势。  
**验证集** 从一开始的 **缓慢下降** ，到后面开始 **反弹** ，这可能是 **过拟合** 的信号。

打印 **验证集** ***loss*** **最低点** 处的模型训练log如下：
```
2025-04-30 21:06:29,082 INFO     pid:1776838 app.train.training_seg:395:logMetrics E55 SegmentationTrainingApp
2025-04-30 21:06:29,083 INFO     pid:1776838 app.train.training_seg:425:logMetrics E55 val      0.7768 loss, 0.0953 precision, 0.7293 recall, 0.1686 f1 score
2025-04-30 21:06:29,084 INFO     pid:1776838 app.train.training_seg:435:logMetrics E55 val_all  0.7768 loss,  72.9% tp,  27.1% fn,     692.4% fp
2025-04-30 21:06:29,122 INFO     pid:1776838 app.train.training_seg:512:saveModel Saved model params to data-unversioned/models/seg/seg_2025-04-30_18.55.29_seg.5500000.state
2025-04-30 21:06:29,125 INFO     pid:1776838 app.train.training_seg:524:saveModel SHA1: 9278dec554cac92a16a6ff3b25ec7bd4eee6f87e
```

***PR*** **曲线图**

![seg-pr.png](../images/seg-pr.png)

从 **PR** 曲线来看，***recall*** 一直 **下降** ，然后突然 **反转上升** ；***precision*** 的趋势刚好与 ***recall*** 相反。

***recall*** 的最 **低** 点刚好对应于 ***precision*** 的最 **高** 点，***recall*** 回升后的 **峰值** 点对应于 ***precision*** 下降后的 **谷**点。

这是符合逻辑的，因为CT影像中结节数据（**Positive样本**）本身 **占比是很小** 的。当预期提升 ***recall*** 时，随着 ***TruePositive*** 的增加，也会引入大量的***FalsePositive*** ，意味着当前模型更倾向于判定为 ***Positive*** 样本。反之亦然，***precision***的提升，也会造成更多的***False Negative***，使得***recall***下降。

关键是找到 **合理的权衡点** ，以满足应用的需求。

根据应用的需求，我们想要**较高的recall**，和**合理的precision**。训练代码中，给 ***fn_loss*** 设置了 **更大** 的权重，以达到惩罚 ***False Negative***，鼓励模型更倾向于作出 ***Positive*** 的预测。
```
return diceLoss_g.mean() + fnLoss_g.mean() * 8
```

我们在 ***recall*** 开始的下降区间（即 ***precision*** 开始的上升区间）选择一些合理的模型：
```
2025-04-30 19:44:56,703 INFO     pid:1776838 app.train.training_seg:395:logMetrics E20 SegmentationTrainingApp
2025-04-30 19:44:56,704 INFO     pid:1776838 app.train.training_seg:425:logMetrics E20 val      0.9284 loss, 0.0274 precision, 0.8225 recall, 0.0531 f1 score
2025-04-30 19:44:56,705 INFO     pid:1776838 app.train.training_seg:435:logMetrics E20 val_all  0.9284 loss,  82.3% tp,  17.7% fn,    2916.0% fp
```
和
```
2025-04-30 20:19:45,367 INFO     pid:1776838 app.train.training_seg:395:logMetrics E35 SegmentationTrainingApp
2025-04-30 20:19:45,368 INFO     pid:1776838 app.train.training_seg:425:logMetrics E35 val      0.8396 loss, 0.0639 precision, 0.7667 recall, 0.1180 f1 score
2025-04-30 20:19:45,369 INFO     pid:1776838 app.train.training_seg:435:logMetrics E35 val_all  0.8396 loss,  76.7% tp,  23.3% fn,    1122.6% fp
```

有趣的是，***loss*** **最低位置** 的模型，刚好是 ***precision*** **峰值**、***recall*** **谷值** 位置处的模型，这意味着模型一开始产生了大量的 ***False Positive*** ，然后模型不断调整以减少 ***False Positive*** 数量，最终在这个位置，对应于我们在loss中设定的损失权重，达到了 **局部最优** 。

**比较** 一下这3个模型：

|Epoch|loss|tp|fn|fp|
|-|-|-|-|-|
|20|0.9284|82.3%|17.7%|2916.0%|
|35|0.8396|76.7%|23.3%|1122.6%|
|55|0.7768|72.9%|27.1%|692.4%|

在后续的实验中，选择 **Epoch-35** 对应的模型作为端到端应用的模型。（也可以选择其他）

# 训练分类模型

### ClsPrepCacheApp 简介

***ClsPrepCacheApp*** 的实现位于 ***src/app/cache/prepcache_cls.py***

为提高训练效率，将 **数据缓存到磁盘** 。  
训练前，请确保已经执行过一次 ***ClsPrepCacheApp***

In [4]:
run(ClsPrepCacheApp, "--num-workers=2")

2025-04-30 23:24:15,945 INFO     pid:1819580 nb:004:run Running: ClsPrepCacheApp(['--num-workers=2']).main()
2025-04-30 23:24:15,948 INFO     pid:1819580 app.cache.prepcache_cls:048:main Starting ClsPrepCacheApp, Namespace(batch_size=1024, num_workers=2)
2025-04-30 23:24:19,101 INFO     pid:1819580 core.dsets_cls:284:__init__ <core.dsets_cls.LunaDataset object at 0x7fa096adb5e0>: 550896 training samples, 549714 neg, 1182 pos, unbalanced ratio
2025-04-30 23:24:19,102 WARNING  pid:1819580 util.util:221:enumerateWithEstimate Stuffing cache ----/538, starting
2025-04-30 23:26:00,863 INFO     pid:1819580 util.util:238:enumerateWithEstimate Stuffing cache   16/538, done at 2025-05-01 00:22:24, 0:58:01
2025-04-30 23:31:40,645 INFO     pid:1819580 util.util:238:enumerateWithEstimate Stuffing cache   64/538, done at 2025-05-01 00:26:23, 1:01:59
2025-05-01 00:05:42,603 INFO     pid:1819580 util.util:238:enumerateWithEstimate Stuffing cache  256/538, done at 2025-05-01 00:51:13, 1:26:49
2025-05-0

### ***ClassificationTrainingApp*** 简介

***ClassificationTrainingApp*** 的实现位于 ***src/app/train/training_cls.py***

#### **Model 架构**

分类任务采用传统的自定义的 ***CNN*** 网络架构，用于处理二分类任务。  
本项目将自定义的 ***CNN*** 架构命名为 ***LunaModel*** 。


***LunaModel*** 的核心组件如下：
- 批归一化层：BatchNorm3d 
- 网络主干：LunaBlock：Conv3d -> ReLu -> Conv3d -> Relu -> MaxPool3d
- 全连接层：Linear -> Softmax（训练时不需要)

***LunaModel*** 架构示意图如下：

![LunaModel(自定义分类Model)架构图](../images/LunaModel.png)

***LunaModel*** 封装了自定义的 ***CNN*** 网络架构的实现，完整的实现位于 ***src/app/model/model_cls.py***

```
class LunaModel(nn.Module):
    def __init__(self, in_channels=1, conv_channels=8):
        super().__init__()

        self.tail_batchnorm = nn.BatchNorm3d(1)

        self.block1 = LunaBlock(in_channels, conv_channels)
        self.block2 = LunaBlock(conv_channels, conv_channels * 2)
        self.block3 = LunaBlock(conv_channels * 2, conv_channels * 4)
        self.block4 = LunaBlock(conv_channels * 4, conv_channels * 8)

        # self.head_linear = nn.Linear(1152, 2)
        self.head_linear = nn.Linear(1152, 2)
        self.head_activation = nn.Softmax(dim=1)

#......................................
#......................................

class LunaBlock(nn.Module):
    def __init__(self, in_channels, conv_channels):
        super().__init__()

        self.conv1 = nn.Conv3d(
            in_channels, conv_channels, kernel_size=3, padding=1, bias=True
        )
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(
            conv_channels, conv_channels, kernel_size=3, padding=1, bias=True
        )
        self.relu2 = nn.ReLU(inplace=True)

        self.maxpool = nn.MaxPool3d(2, 2)

#......................................
#......................................
```

#### **输入/输出**

输入：形如 ***(C=1, D=32, H=48, W=48)*** 的 *3D* 数据块

输出：形如 ***(C=2)*** ，*C\[0\]* 表示归属于非结节的概率， *C\[1\]* 表示归属于结节的概率

#### **其他**

数据集标签位于 ***data/luna/*** 目录下

***annotations.csv/annotations_with_malignancy.csv*** 文件为专业医生标注的文件，全部为结节。

***candidates.csv*** 文件为传统结节检测算法自动生成的文件，其中含有疑似但不是结节的数据，可提供阴性案例用于训练分类器。

In [4]:
run(ClassificationTrainingApp, "--num-workers=1", "--epochs=100", "--tb-prefix=nodule-cls", "nodule_cls")

2025-05-01 10:27:09,103 INFO     pid:1822502 nb:004:run Running: ClassificationTrainingApp(['--num-workers=1', '--epochs=100', '--tb-prefix=nodule-cls', 'nodule_cls']).main()
2025-05-01 10:27:09,180 INFO     pid:1822502 app.train.training_cls:143:initModel Using CUDA; 1 devices.
2025-05-01 10:27:10,367 INFO     pid:1822502 app.train.training_cls:209:main Starting ClassificationTrainingApp, Namespace(batch_size=24, num_workers=1, epochs=100, dataset='LunaDataset', model='LunaModel', malignant=False, finetune='', finetune_depth=1, tb_prefix='nodule-cls', comment='nodule_cls')
2025-05-01 10:27:14,046 INFO     pid:1822502 core.dsets_cls:284:__init__ <core.dsets_cls.LunaDataset object at 0x7faedf6c2dc0>: 499313 training samples, 498285 neg, 1028 pos, 1:1 ratio
2025-05-01 10:27:14,187 INFO     pid:1822502 core.dsets_cls:284:__init__ <core.dsets_cls.LunaDataset object at 0x7fad6cd03a00>: 51583 validation samples, 51429 neg, 154 pos, unbalanced ratio
2025-05-01 10:27:14,188 INFO     pid:182250

KeyboardInterrupt: 

#### **基于 Tensorboard 的效果展示**

完整的 **Metrics** 见 ***runs/nodule-cls/2025-05-01_10.27.09-trn_cls-nodule_cls/*** 和 ***runs/nodule-cls/2025-05-01_10.27.09-val_cls-nodule_cls/***

**训练集** 数据标注为 **蓝色**  
**验证集** 数据标注为 **红色**  
（平滑系数 *0.3*）

***Loss*** **趋势图**

![nodule-cls-loss.png](../images/nodule-cls-loss.png)

从 ***loss*** 来看， **训练集** 和 **验证集** 一直呈 **下降** 趋势，末尾处趋于平缓。

***PR*** **曲线图**

![nodule-cls-pr.png](../images/nodule-cls-pr.png)

从 **PR** 曲线来看，***recall***, ***precision***, ***f1_score*** 一直稳步 **上升** ，末尾趋于平缓。

**提示**：从上图来看，可以 **继续训练** ，查看指标是否进一步改善。由于在训练过程中查看log发现当前模型已经 **足够满足演示需求** ，所以用 ***ctrl+c*** 提前终止了训练。

取 ***Epoch-20*** 的模型作为最佳模型，集成到端到端的结节检测应用中。模型的训练log如下：

```
2025-05-01 14:49:10,416 INFO     pid:1822502 app.train.training_cls:346:logMetrics E20 ClassificationTrainingApp
2025-05-01 14:49:10,442 INFO     pid:1822502 app.train.training_cls:419:logMetrics E20 val      0.0534 loss,  98.6% correct, 0.1682 precision, 0.9675 recall, 0.2865 f1 score, 0.9898 auc
2025-05-01 14:49:10,442 INFO     pid:1822502 app.train.training_cls:432:logMetrics E20 val_neg  0.0532 loss,  98.6% correct (50692 of 51429)
2025-05-01 14:49:10,443 INFO     pid:1822502 app.train.training_cls:443:logMetrics E20 val_pos  0.1440 loss,  96.8% correct (149 of 154)
2025-05-01 14:49:10,726 DEBUG    pid:1822502 app.train.training_cls:548:saveModel Saved model params to data-unversioned/models/nodule-cls/cls_2025-05-01_10.27.09_nodule_cls.2000000.state
2025-05-01 14:49:10,738 DEBUG    pid:1822502 app.train.training_cls:564:saveModel Saved model params to data-unversioned/models/nodule-cls/cls_2025-05-01_10.27.09_nodule_cls.best.state
2025-05-01 14:49:10,744 INFO     pid:1822502 app.train.training_cls:567:saveModel SHA1: e34707c14ce0220625113c6bbf1407caaecfcd2a
```

TODO: 补充ROC视图选择最佳阈值

# 集成各模型以实现端到端的结节检测

### ***NoduleAnalysisApp*** 简介

***NoduleAnalysisApp*** 的实现位于 ***src/app/infer/nodule_analysis.py***

#### **流程**

***NoduleAnalysisApp*** 的核心步骤如下：
1. 下载 *LUNA16* 的数据集，并封装到 *Dataset*
2. 定义 **分割** 模型架构，并训练出有效模型备用
3. 将分割模型输出 **转换** 成分类模型的有效输入
4. 定义 **分类** 模型架构，并训练出有效模型备用
5. **串联** 2、3、4步骤，并输出诊断结果（ ***confusion matrix*** ）

***NoduleAnalysisApp*** 完整流程示意图如下：

![完整的端到端工程的流程图](../images/end-to-end-flow.png)

#### **输入/输出**

输入：完整的CT扫描影像

输出：***Confusion Matrix***，示例如下：
```
# 分割模型漏掉11个结节；分类模型错误筛除2个结节；成功检查出141个结节
Total
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |            93908 |             2901
          Benign |               11 |                0 |               91
       Malignant |                0 |                2 |               50
```

#### **备注**

本实验并没有另外训练一个专门分类 **良性/恶性** 结节的分类器  
TODO: 在结节/非结节分类器基础上，微调一个良性/恶性结节分类器

步骤3的分割输出转换为分类输入，主要是

1. 基于 **连通域** 作聚类分析
2. 同一连通域内通过 **加权平均** 计算质心位置（权重为强度值），记为结节中心

代码实现如下：

```
def clusterSegmentationOutput(self, series_uid,  ct, clean_g):
    clean_a = clean_g.cpu().numpy()
    candidateLabel_a, candidate_count = measure.label(clean_a)
    centerIrc_list = measure.center_of_mass(
        ct.hu_a.clip(-1000, 1000) + 1001,
        labels=candidateLabel_a,
        index=list(range(1, candidate_count+1)),
    )
```

In [4]:
run(NoduleAnalysisApp, "--num-workers=1", "--run-validation",
    "--classification-path=data-unversioned/models/nodule-cls/cls_2025-05-01_10.27.09_nodule_cls.best.state",
   "--segmentation-path=data-unversioned/models/seg/seg_2025-04-30_18.55.29_seg.3500000.state"
   )

2025-05-05 13:14:24,424 INFO     pid:65182 nb:004:run Running: NoduleAnalysisApp(['--num-workers=1', '--run-validation', '--classification-path=data-unversioned/models/nodule-cls/cls_2025-05-01_10.27.09_nodule_cls.best.state', '--segmentation-path=data-unversioned/models/seg/seg_2025-04-30_18.55.29_seg.3500000.state']).main()
2025-05-05 13:14:24,426 DEBUG    pid:65182 app.infer.nodule_analysis:295:initModels data-unversioned/models/seg/seg_2025-04-30_18.55.29_seg.3500000.state
2025-05-05 13:14:25,027 DEBUG    pid:65182 app.infer.nodule_analysis:311:initModels data-unversioned/models/nodule-cls/cls_2025-05-01_10.27.09_nodule_cls.best.state
2025-05-05 13:14:25,063 INFO     pid:65182 app.infer.nodule_analysis:371:main Starting NoduleAnalysisApp, Namespace(batch_size=4, num_workers=1, run_validation=True, include_train=False, segmentation_path='data-unversioned/models/seg/seg_2025-04-30_18.55.29_seg.3500000.state', cls_model='LunaModel', classification_path='data-unversioned/models/nodule-

数据集中的结节与多个分割后的结节相匹配
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |              993 |               32
          Benign |                1 |                0 |                0
       Malignant |                0 |                0 |                1


2025-05-05 13:14:46,816 INFO     pid:65182 src.core.dsets_seg:273:__init__ <src.core.dsets_seg.Luna2dSegmentationDataset object at 0x7fb28c2c1880>: 1 general series, 143 slices, 1 nodules
2025-05-05 13:14:51,728 INFO     pid:65182 src.core.dsets_cls:284:__init__ <src.core.dsets_cls.LunaDataset object at 0x7fb3e00d6fa0>: 443 training samples, 443 neg, 0 pos, unbalanced ratio


1.3.6.1.4.1.14519.5.2.1.6279.6001.102681962408431413578140925249
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |              423 |               19
          Benign |                0 |                0 |                0
       Malignant |                0 |                0 |                1


2025-05-05 13:14:53,624 INFO     pid:65182 src.core.dsets_seg:273:__init__ <src.core.dsets_seg.Luna2dSegmentationDataset object at 0x7fb28c266610>: 1 general series, 328 slices, 1 nodules
2025-05-05 13:15:04,569 INFO     pid:65182 src.core.dsets_cls:284:__init__ <src.core.dsets_cls.LunaDataset object at 0x7fb3f1cd5b80>: 975 training samples, 975 neg, 0 pos, unbalanced ratio


...

1.3.6.1.4.1.14519.5.2.1.6279.6001.910607280658963002048724648683
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             1528 |               15
          Benign |                0 |                0 |                1
       Malignant |                0 |                0 |                0


2025-05-05 13:33:33,432 INFO     pid:65182 src.core.dsets_seg:273:__init__ <src.core.dsets_seg.Luna2dSegmentationDataset object at 0x7fb3f25048e0>: 1 general series, 481 slices, 1 nodules
2025-05-05 13:33:59,375 INFO     pid:65182 src.core.dsets_cls:284:__init__ <src.core.dsets_cls.LunaDataset object at 0x7fb28c2c1610>: 2010 training samples, 2010 neg, 0 pos, unbalanced ratio


数据集中的结节与多个分割后的结节相匹配
1.3.6.1.4.1.14519.5.2.1.6279.6001.946129570505893110165820050204
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             1967 |               41
          Benign |                0 |                0 |                0
       Malignant |                0 |                0 |                1


2025-05-05 13:34:04,248 INFO     pid:65182 src.core.dsets_seg:273:__init__ <src.core.dsets_seg.Luna2dSegmentationDataset object at 0x7fb28c280c70>: 1 general series, 145 slices, 2 nodules
2025-05-05 13:34:09,538 INFO     pid:65182 src.core.dsets_cls:284:__init__ <src.core.dsets_cls.LunaDataset object at 0x7fb269a5aee0>: 1077 training samples, 1077 neg, 0 pos, unbalanced ratio
2025-05-05 13:34:11,849 WARNING  pid:65182 util.util:251:enumerateWithEstimate Series ----/89, done at 2025-05-05 13:34:11
2025-05-05 13:34:11,884 INFO     pid:65182 nb:006:run Finished: NoduleAnalysisApp.['--num-workers=1', '--run-validation', '--classification-path=data-unversioned/models/nodule-cls/cls_2025-05-01_10.27.09_nodule_cls.best.state', '--segmentation-path=data-unversioned/models/seg/seg_2025-04-30_18.55.29_seg.3500000.state']).main()


1.3.6.1.4.1.14519.5.2.1.6279.6001.970428941353693253759289796610
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             1043 |               32
          Benign |                0 |                0 |                0
       Malignant |                0 |                1 |                1
Total
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |            93908 |             2901
          Benign |               11 |                0 |               91
       Malignant |                0 |                2 |               50

未被分割出来的结节:
DebugInfo(series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860', labeled_centre_xyz=(103.78365090000001, -211.92514869999997, -227.12125), segmented_centre_xyz=(0, 0, 0), segmented_centre_irc=(0, 0, 0), pred_nodule=0)
DebugInfo(series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.144883090372691745980459537